In [26]:
import os

In [27]:
%pwd

'/mnt/f/DS_Tech_Stack/MLOps'

In [33]:
os.chdir('Chest-Cancer-Classification-using-MLflow-DVC')

In [34]:
%pwd

'/mnt/f/DS_Tech_Stack/MLOps/Chest-Cancer-Classification-using-MLflow-DVC'

## Entity

In [42]:
from dataclasses import dataclass
from pathlib import Path

#create custom Return type
@dataclass(frozen=True)
class DataIngestionConfig:
    """
    represent the configuration required for data ingestion tasks
    """
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

## Config

In [43]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [44]:
class ConfigurationManager:
    """
    manage the configuration settings required for data ingestion
    """
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        #read configuration settings from YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        #handles data ingestion process including downloading and extracting data files
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir,
                                                    source_URL=config.source_URL,
                                                    local_data_file=config.local_data_file,
                                                    unzip_dir=config.unzip_dir)
        return data_ingestion_config

## Components

In [45]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [50]:
class DataIngestion:
    """
    handle the data ingestion process

    Args:
        config(DataIngestionConfig): DataIngestionConfig object
    """
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self) -> str:
        """
        fetch data from remote URL
        """
        try:
            #fetch data
            dataset_url = self.config.source_URL
            #save downloaded file to the local data file path
            zip_download_dir = self.config.local_data_file
            os.makedirs('artifacts/data_ingestion', exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        extracts the zip file into the data dir
        func returns None
        """
        unzip_path = self.config.unzip_dir
        #create unzip dir if it doesn't exist
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

## Pipeline

In [51]:
try:
    #initialize obj to read configuration
    config = ConfigurationManager()
    
    #retrieve the data ingestion config using the resp method
    data_ingestion_config = config.get_data_ingestion_config()

    #create instance of DataIngestion class with the above config
    data_ingestion = DataIngestion(config=data_ingestion_config)

    #perform the data ingestion tasks
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-04-01 23:46:11,434: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-01 23:46:11,444: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-01 23:46:11,445: INFO: common: created directory at: artifacts]
[2024-04-01 23:46:11,447: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-01 23:46:11,448: INFO: 2336601768: Downloading data from https://drive.google.com/file/d/1bfhjpjUK-fqTwzKv50BLuMv8NWvv2agV/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1bfhjpjUK-fqTwzKv50BLuMv8NWvv2agV
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1bfhjpjUK-fqTwzKv50BLuMv8NWvv2agV&confirm=t&uuid=bf5f04af-f2b7-4ed5-ac74-f3c495834bce
To: /mnt/f/DS_Tech_Stack/MLOps/Chest-Cancer-Classification-using-MLflow-DVC/artifacts/data_ingestion/data.zip
100%|██████████| 124M/124M [00:09<00:00, 12.8MB/s] 

[2024-04-01 23:46:23,189: INFO: 2336601768: Downloaded data from https://drive.google.com/file/d/1bfhjpjUK-fqTwzKv50BLuMv8NWvv2agV/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
